In [14]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models, CorruptionTypes
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [15]:
gold_standard = pd.read_csv("../datasets/llm_dataset/Radiology_modality_sample.csv")[:10]

# Duplicate and append the DataFrame 5 times
gold_standard_extended = pd.concat([gold_standard.copy() for _ in range(5)], ignore_index=True)
gold_standard_extended = pd.concat([group for _, group in gold_standard_extended.groupby("dicom_uid")], ignore_index=True)
print(gold_standard_extended)

                        dicom_uid  rows  columns        series_desc modality
0   1.2.840.166348.444565830.2739  1024     1024  PELVIS_ULTRASOUND       US
1   1.2.840.166348.444565830.2739  1024     1024  PELVIS_ULTRASOUND       US
2   1.2.840.166348.444565830.2739  1024     1024  PELVIS_ULTRASOUND       US
3   1.2.840.166348.444565830.2739  1024     1024  PELVIS_ULTRASOUND       US
4   1.2.840.166348.444565830.2739  1024     1024  PELVIS_ULTRASOUND       US
5    1.2.840.178211.688829522.637  2048     2048        ABDOMEN_KUB       XR
6    1.2.840.178211.688829522.637  2048     2048        ABDOMEN_KUB       XR
7    1.2.840.178211.688829522.637  2048     2048        ABDOMEN_KUB       XR
8    1.2.840.178211.688829522.637  2048     2048        ABDOMEN_KUB       XR
9    1.2.840.178211.688829522.637  2048     2048        ABDOMEN_KUB       XR
10  1.2.840.285501.600086396.5032   512      512      ABDOMEN_W_CON       CT
11  1.2.840.285501.600086396.5032   512      512      ABDOMEN_W_CON       CT

In [16]:
corrupted_datasets = Pipeline.generate_corrupted_datasets(
    dataset=gold_standard_extended,
    cell_corruption_types=[CorruptionTypes.CellCorruptionTypes.NULL, CorruptionTypes.CellCorruptionTypes.OUTLIER],
    row_corruption_types=[],
    columns_to_exclude=["dicom_uid"],
    severity=0.15,
    output_size=1
)


In [17]:
merged_df = Pipeline.merge_with_llm(dataset=corrupted_datasets[0], primary_key="dicom_uid",model_name=Models.GeminiModels.GEMINI_2_0_FLASH)


In [18]:


stats_micro = Pipeline.evaluate_micro(gold_standard=gold_standard_extended, cleaned_dataset=merged_df, corrupted_dataset=corrupted_datasets[0])
pprint(stats_micro)

{'accuracy': 1.0,
 'column_names': ['dicom_uid', 'rows', 'columns', 'series_desc', 'modality'],
 'f1_score': 1.0,
 'false_negative': 0,
 'false_negative_rate': 0.0,
 'false_positive': 0,
 'false_positive_rate': 0.0,
 'num_columns': 5,
 'num_rows': 50,
 'precision': 1.0,
 'recall': 1.0,
 'true_negative': 220,
 'true_positive': 30}
